In [1]:
import os
import cv2
import numpy as np
import shutil
import random
from imgaug import augmenters as iaa

In [2]:
rotate90 = iaa.Rotate(90)

rotate180 = iaa.Rotate(180)

rotate270 = iaa.Rotate(270)

random_rotate = iaa.Affine(rotate=(-25, 25))

crop = iaa.Crop(px=(0, 15))

scale = iaa.Affine(scale=(0.8, 0.95))

TranslateXY = iaa.Affine(translate_px={"x": (-20, 20), "y": (-20, 20)})

In [3]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Path to the dataset file in Google Drive
dataset_path_drive = '/content/drive/MyDrive/Project_482/train.zip'

# Path to the destination directory in Colab
dataset_path_colab = '/content/train.zip'

# Copy the dataset file from Google Drive to Colab
!cp "{dataset_path_drive}" "{dataset_path_colab}"


In [5]:
!unzip /content/train.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/AD/AD_2667.png    
  inflating: train/AD/AD_2671.png    
  inflating: train/AD/AD_2672.png    
  inflating: train/AD/AD_2673.png    
  inflating: train/AD/AD_2676.png    
  inflating: train/AD/AD_2679.png    
  inflating: train/AD/AD_2682.png    
  inflating: train/AD/AD_2686.png    
  inflating: train/AD/AD_2687.png    
  inflating: train/AD/AD_2695.png    
  inflating: train/AD/AD_2700.png    
  inflating: train/AD/AD_2702.png    
  inflating: train/AD/AD_2703.png    
  inflating: train/AD/AD_2705.png    
  inflating: train/AD/AD_2706.png    
  inflating: train/AD/AD_2707.png    
  inflating: train/AD/AD_2709.png    
  inflating: train/AD/AD_2716.png    
  inflating: train/AD/AD_2719.png    
  inflating: train/AD/AD_2721.png    
  inflating: train/AD/AD_2722.png    
  inflating: train/AD/AD_2723.png    
  inflating: train/AD/AD_2731.png    
  inflating: train/AD/AD_2733.png    
  inflating: train/AD/AD_2741.png    

In [6]:
def start_process(directory_path):
    number_of_image_per_folder = []
    for foldername in os.listdir(directory_path):
        number_of_image_per_folder.append(len(os.listdir(directory_path+'/'+foldername+'/')))
    max_val = max(number_of_image_per_folder)
    #print(max_val)
    count = 0
    for foldername in os.listdir(directory_path):
        src_folder = directory_path+'/'+foldername+'/'
        n_diff = max_val - len(os.listdir(src_folder))
        print(n_diff)
        dst_folder = '/content/DF/'+ foldername+'/'
        try:
            os.mkdir(dst_folder)
        except:
            pass
        for f in os.listdir(src_folder):
            shutil.copy2(src_folder+f, dst_folder+f)

        #Randomly select image from old folder
        for i in range(n_diff):
            image_files = os.listdir(src_folder)
            random_image_file = random.choice(image_files)
            random_image_path = os.path.join(src_folder, random_image_file)
            #Now apply augmentation to this image randomly
            image = cv2.imread(random_image_path)
            image = cv2.resize(image, (630,630))
            random_number = random.randint(1,14)
            if random_number == 1:
                augmented_image = rotate90.augment_image(image)

            elif random_number == 2:
                augmented_image = rotate180.augment_image(image)

            elif random_number == 3:
                augmented_image = rotate270.augment_image(image)

            elif random_number == 4:
                augmented_image = random_rotate.augment_image(image)

            elif random_number == 5:
                augmented_image = crop.augment_image(image)

            elif random_number == 6:
                augmented_image = iaa.fliplr(image)

            elif random_number == 7:
                augmented_image = iaa.flipud(image)

            elif random_number == 8:
                augmented_image = scale.augment_image(image)

            elif random_number == 9:
                augmented_image = TranslateXY.augment_image(image)

            elif random_number == 10:
                cropped_image = crop.augment_image(image)
                augmented_image = random_rotate.augment_image(cropped_image)

            elif random_number == 11:
                horizontal_flip = iaa.fliplr(image)
                augmented_image = random_rotate.augment_image(horizontal_flip)

            elif random_number == 12:
                vertical_flip = iaa.flipud(image)
                augmented_image = random_rotate.augment_image(vertical_flip)

            elif random_number == 13:
                scaled_image = scale.augment_image(image)
                augmented_image = random_rotate.augment_image(scaled_image)

            else:
                xy_translated = TranslateXY.augment_image(image)
                augmented_image = random_rotate.augment_image(xy_translated)


            #place the augmented image to the new folder
            count += 1
            cv2.imwrite(dst_folder+str(count)+'a.jpg', augmented_image)
            pass
        pass
    pass


In [7]:
start_process('/content/train')

472
2705
0


In [8]:
import zipfile
import os

# Folder to zip
folder_to_zip = '/content/DF'

# Destination zip file
zip_file_path = '/content/DF.zip'

# Create a ZipFile object
with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Iterate over all the files in the folder
    for root, _, files in os.walk(folder_to_zip):
        for file in files:
            # Add each file to the zip
            zipf.write(os.path.join(root, file),
                       os.path.relpath(os.path.join(root, file),
                                       os.path.join(folder_to_zip, '..')))

print("Folder zipped successfully!")


Folder zipped successfully!
